In [1]:
## COMP9517 Computer Vision Project 20T2
# code: Main program for the project
#
# Group C:
# Connor Baginski (z5207788)
# Bhumika Singhal (z5234799)
# Rishav Guha (z5294757)
# Amel Johny (z5294308)


# from cell_tracking.kalmanFilter import KalmanFilter
# from scipy.optimize import linear_sum_assignment
# # from cell_tracking.tracking_objects  import Tracker 
# from cell_tracking.tracking_objects  import Track 
from cell_tracking.helpers import fi_list, find_labels, find_centers, plot_rectangles,draw_path
from cell_tracking.pathTracking import PathTracker

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import cv2
import os
import imutils

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from skimage.morphology import watershed, disk
from skimage.feature import peak_local_max
from skimage.filters import meijering

from scipy import ndimage as ndi


def fi_list(path):
    """
    Return a sorted list of filenames in a given path
    """
    return sorted([os.path.join(path, f) for f in os.listdir(path)])

def custom_thresh(image):
    for r in range(image.shape[0]):
        for c in range(image.shape[1]):
            if image[r,c] > 0.05:
                image[r,c] = 255
            else:
                image[r,c] = 0

    return image

def disk_erode(img, radius=24, iters=1):
    image = img.copy()
    kern_disk = disk(radius)
    eroded = cv2.erode(image, kern_disk, iterations=iters)
    
    return eroded


In [2]:
def detect_DIC():
    for filename in fi_list('DIC-C2DH-HeLa/Sequence 1 Masks'):
        if not filename.endswith(".tif"):
            continue
        print(filename)
        image = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        plt.title("Ground Truth")
        plt.imshow(image)
        plt.show()
        
        cell_mask = (image > 0).astype(np.uint8)
        plt.imshow(cell_mask)
        plt.show()
        
        markers = disk_erode(cell_mask)
        plt.title("Markers")
        plt.imshow(markers)
        plt.show()
        """
        blurred = cv2.medianBlur(image, 21)
        
        denoised = cv2.fastNlMeansDenoising(blurred, h=3)        
        
        # adaptive normalization - see paper https://is.muni.cz/www/svoboda/ISBI-final.pdf
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
        cl1 = clahe.apply(denoised)
        
        mj_filtered = meijering(cl1)
        mj_filtered = custom_thresh(mj_filtered)
        
        kernel = np.ones((4,4),np.uint8)

        eroded = cv2.erode(mj_filtered, kernel, iterations=1)

        thresh = cv2.threshold(eroded, 129, 255, cv2.THRESH_BINARY)[1]

        distance = ndi.distance_transform_edt(thresh)
        local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((10, 10)),
                                labels=thresh)
        markers, _ = ndi.label(local_maxi)
        ws_labels = watershed(-distance, markers, mask=thresh)
        
        plt.gray()
        plt.imshow(image)
        plt.show()
        
        plt.imshow(blurred)
        plt.show()
        
        plt.imshow(denoised)
        plt.show()
        
        plt.imshow(cl1)
        plt.show()
        
        plt.imshow(mj_filtered)
        plt.show()
        
        plt.imshow(eroded)
        plt.show()
        
        plt.imshow(thresh, cmap='gray')
        plt.show()
        
        plt.imshow(ws_labels)
        plt.show()
        
        #break stops loop after one iteration for debugging
        break
        """
    return

In [3]:
# def detect_Fluo():
#     #initialising variables for tracking
#     tracker = Tracker(160, 30, 5, 100)
#     skip_frame_count = 0
#     trackIdCount = 0
#     for filename in fi_list('Fluo-N2DL-HeLa/Sequence 1'):
#         if not filename.endswith(".tif"):
#             continue
#         # print(filename)
#         image = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
#         # Threshold at value of 129
#         thresh = cv2.threshold(image, 129, 255, cv2.THRESH_BINARY)[1]
#         distance = ndi.distance_transform_edt(thresh)
#         local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((10, 10)),
#                                 labels=thresh)
#         markers, _ = ndi.label(local_maxi)
#         ws_labels = watershed(-distance, markers, mask=thresh)
#         print(len(ws_labels))
        
#         #TODO: cite/recode following
#         # loop over the unique labels returned by the Watershed
#         # algorithm
#         centers = []
#         for label in np.unique(ws_labels):
#             # if the label is zero, we are examining the 'background'
#             # so simply ignore it
#             if label == 0:
#                 continue

#             # otherwise, allocate memory for the label region and draw
#             # it on the mask
#             mask = np.zeros(image.shape, dtype="uint8")
#             mask[ws_labels == label] = 255

#             # detect contours in the mask and grab the largest one
#             cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
#                 cv2.CHAIN_APPROX_SIMPLE)
#             cnts = imutils.grab_contours(cnts)
#             c = max(cnts, key=cv2.contourArea)

#             # draw a rectangle enclosing the object
#             try:
#                 x,y,w,h = cv2.boundingRect(c)
#                 center = np.array([[int(x + w / 2.0)], [int(y + h / 2.0)]])
#                 centers.append(center)
#                 # print(bounding_box_center)
                
#             except ZeroDivisionError:
#                 pass
#             cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
#             cv2.putText(image, "#{}".format(label), (int(x) - 10, int(y)),
#                 cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
#         image = track_path(image,centers,trackIdCount,tracker)
#         # plt.gray()
#         # plt.imshow(thresh, cmap='gray')
#         # plt.show()
#         plt.imshow(image)
#         plt.show()
#     return



In [4]:

def detect_PhC():
    pathTracker = PathTracker()
    for filename in fi_list('PhC-C2DL-PSC/Sequence 1'):
        if not filename.endswith(".tif"):
            continue
        #process images to segment the cells to find labels
        ws_lables,image = find_labels(filename)
        #Extract the centers of cells detected
        centers, boundingBoxes = find_centers(ws_lables,image)
        #draw bounding boxes for the detected cells
        plot_rectangles(image, centers, boundingBoxes)
                    
        pathTracker.update(image,centers)
#         plt.imshow(opening, cmap='gray')
#         plt.show()
        # plt.imshow(ws_labels, cmap='gray')
        # plt.show()
    return

In [5]:
def main():
    select = int(input("Choose a dataset.\n1) DIC-C2DH-HeLa\n2) Fluo-N2DL-HeLa\n3) PhC-C2DL-PSC\n> "))
    # select = 3 ##FOR TESTING

    if select == 1:
        detect_DIC()
    elif select == 2:
        detect_Fluo()
    elif select == 3:
        detect_PhC()
    else:
        print("Invalid input.")

if __name__ == '__main__':
    main()

49
24
94
111
81
106
24
49
81
94
106
111
2
23
91
29
99
79
2
23
29
79
91
99
8
23
29
34
82
95
79
25
21
24
72
80
8
21
23
24
25
29
34
72
79
80
82
95
14
16
23
47
86
90
111
91
64
14
16
23
47
64
86
90
91
111
5
53
93
111
91
5
53
91
93
111
19
17
7
29
24
36
102
87
33
58
104
64
125
39
7
17
19
21
24
25
27
29
30
33
36
39
58
64
87
102
104
125
84
105
86
84
86
105
22
16
103
51
29
16
22
29
51
103
80
70
48
33
50
114
45
91
93
110
33
45
48
50
70
80
91
93
110
114
21
26
36
17
89
95
39
108
17
21
26
36
39
89
95
108
34
83
84
86
80
104
34
80
83
84
86
104
14
68
36
86
2
87
109
105
2
14
36
68
86
87
105
109
41
32
4
69
109
87
4
32
41
69
87
109
